In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import tensorflow as tf
import os.path as op

In [3]:
log_dir = './logs'
if tf.gfile.Exists(log_dir):
    tf.gfile.DeleteRecursively(log_dir)


In [4]:
def read_and_decode(filename):
    filename_queue = tf.train.string_input_producer([filename],
                                                    num_epochs=None)

    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    features = tf.parse_single_example(
        serialized_example,
        features={
                'image/height': tf.FixedLenFeature([], tf.int64),
                'image/width': tf.FixedLenFeature([], tf.int64),
                'image/depth': tf.FixedLenFeature([], tf.int64),
                'label': tf.FixedLenFeature([], tf.int64),
                'image/raw': tf.VarLenFeature(tf.string)})

    # Shape elements must be int32 tensors!
    height = tf.cast(features['image/height'], tf.int32)
    width = tf.cast(features['image/width'], tf.int32)
    depth = tf.cast(features['image/depth'], tf.int32)
    
    # Decode the image from its raw representation:
    image = tf.decode_raw(features['image/raw'].values, tf.uint8)

    # Reshape it back to its original shape:
    im_shape = tf.pack([height, width, depth])
    image = tf.reshape(image, im_shape)
    #tf.random_crop(image, [height, width, depth])
    # Convert from [0, 255] -> [0, 1] floats.
    image = tf.cast(image, tf.float32) * (1. / 255)

    # Convert label from a scalar uint8 tensor to an int32 scalar.
    label = tf.cast(features['label'], tf.int32)
    return image, label

In [5]:
image, label = read_and_decode("../data/cells_train.tfrecords")
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
tf.train.start_queue_runners(sess=sess)
im_1, lab_1 = sess.run([image, label])

In [6]:
image, label = read_and_decode("../data/cells_train.tfrecords")

with tf.name_scope('input'):
    images_batch, labels_batch = tf.train.shuffle_batch(
        [image, label], batch_size=40,
        capacity=400,
        shapes=(im_1.shape, lab_1.shape),
        min_after_dequeue=200)

In [7]:
def weight_variable(shape, fan_in, fan_out):
    """ 
    Initialize weights with the Xavier initialization
    """
    low = -4*np.sqrt(6.0/(fan_in + fan_out)) 
    high = 4*np.sqrt(6.0/(fan_in + fan_out))
    return tf.Variable(tf.random_uniform(shape, minval=low, maxval=high, dtype=tf.float32))

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [8]:
def variable_summaries(var):
    """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)


In [9]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

In [10]:
with tf.name_scope("First_convolutional_layer"):
    with tf.name_scope("Weights"):
        W_conv1 = weight_variable([5, 5, 3, 32], 256*256*3, 32)
        variable_summaries(W_conv1)
    with tf.name_scope("Bias"):
        b_conv1 = bias_variable([32])
        variable_summaries(b_conv1)
    with tf.name_scope('Preactivation'):
        preact1 = conv2d(images_batch, W_conv1) + b_conv1
        tf.summary.histogram('preactivation', preact1)

    h_conv1 = tf.nn.relu(preact1)
    tf.summary.histogram("convolution", h_conv1)
    with tf.name_scope("Pooling"):
        h_pool1 = max_pool_2x2(h_conv1)

Second convolutional layer:

In [11]:
with tf.name_scope("Second_convolutional_layer"):
    with tf.name_scope("Weights"):
        W_conv2 = weight_variable([5, 5, 32, 64], 128*128*32, 64)
        variable_summaries(W_conv2)
    with tf.name_scope("Bias"):
        b_conv2 = bias_variable([64])
        variable_summaries(b_conv2)
    with tf.name_scope("Preactivation"):
        preact2 = conv2d(h_pool1, W_conv2) + b_conv2
        tf.summary.histogram('preactivation', preact2)
    h_conv2 = tf.nn.relu(preact2)
    tf.summary.histogram("convolution", h_conv2)
    with tf.name_scope("Pooling"):
        h_pool2 = max_pool_2x2(h_conv2)

In [12]:
with tf.name_scope("Fully_connected_layer"):
    with tf.name_scope("Weights"):
        W_fc1 = weight_variable([64 * 64 * 64, 1024], 64*64*64, 1024)
        variable_summaries(W_fc1)
    with tf.name_scope("Bias"):
        b_fc1 = bias_variable([1024])
        variable_summaries(b_fc1)
        
    h_pool2_flat = tf.reshape(h_pool2, [-1, 64 * 64 * 64])
    with tf.name_scope("Preactivation"):
        preact3 = tf.matmul(h_pool2_flat, W_fc1) + b_fc1
        variable_summaries(preact3)
    
    h_fc1 = tf.nn.relu(preact3)
    tf.summary.histogram('Fully_connected_output', h_fc1)

In [13]:
with tf.name_scope("Dropout"):
    keep_prob = tf.placeholder(tf.float32)
    tf.summary.scalar('Keep_Probability', keep_prob)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [14]:
W_fc2 = weight_variable([1024, 3], 1024, 3)
b_fc2 = bias_variable([3])
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [15]:
with tf.name_scope("Loss"):
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(y_conv, labels_batch)
    loss_mean = tf.reduce_mean(loss)
    tf.summary.scalar('Mean_loss', loss_mean)
with tf.name_scope("Train"):
    train_op = tf.train.AdamOptimizer(1e-5).minimize(loss_mean)

In [16]:
y_pred = tf.cast(tf.argmax(y_conv, 1), tf.int32)

In [17]:
with tf.name_scope("Accuracy"):
    with tf.name_scope("Correct_prediction"):
        correct_prediction = tf.equal(y_pred, labels_batch)
    with tf.name_scope("Accuracy"):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.summary.scalar('Accuracy', accuracy)


In [18]:
sess = tf.Session()

In [19]:
merged = tf.summary.merge_all()
log_dir = './logs'
train_writer = tf.summary.FileWriter(op.join(log_dir, 'train'), sess.graph)
test_writer = tf.summary.FileWriter(op.join(log_dir, 'test'))

In [20]:
init = tf.global_variables_initializer()
sess.run(init)
tf.train.start_queue_runners(sess=sess)

[<Thread(Thread-6, started daemon 123145603739648)>,
 <Thread(Thread-7, started daemon 123145608994816)>,
 <Thread(Thread-8, started daemon 123145614249984)>]

In [21]:
max_steps = 2000
for i in range(max_steps):
    if i % 10 == 0:  # Record summaries and test-set accuracy
        summary, acc = sess.run([merged, accuracy], feed_dict={keep_prob: 1.0})
        test_writer.add_summary(summary, i)
        print('Accuracy at step %s: %s' % (i, acc))
    else:  # Record train set summaries, and train
        if i % 100 == 99:  # Record execution stats
            run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
            run_metadata = tf.RunMetadata()
            summary, _ = sess.run([merged, train_op],
                                  feed_dict={keep_prob: 0.5},
                                  options=run_options,
                                  run_metadata=run_metadata)
            train_writer.add_run_metadata(run_metadata, 'step%03d' % i)
            train_writer.add_summary(summary, i)
            print('Adding run metadata for', i)
        else:  # Record a summary
            summary, _ = sess.run([merged, train_op], feed_dict={keep_prob: 0.5})
            train_writer.add_summary(summary, i)
train_writer.close()
test_writer.close()


Accuracy at step 0: 0.425
Accuracy at step 10: 0.375
Accuracy at step 20: 0.45
Accuracy at step 30: 0.35
Accuracy at step 40: 0.375
Accuracy at step 50: 0.35
Accuracy at step 60: 0.325
Accuracy at step 70: 0.3
Accuracy at step 80: 0.45
Accuracy at step 90: 0.7
Adding run metadata for 99
Accuracy at step 100: 0.25
Accuracy at step 110: 0.3
Accuracy at step 120: 0.45
Accuracy at step 130: 0.4
Accuracy at step 140: 0.4
Accuracy at step 150: 0.675
Accuracy at step 160: 0.35
Accuracy at step 170: 0.425
Accuracy at step 180: 0.7
Accuracy at step 190: 0.5
Adding run metadata for 199
Accuracy at step 200: 0.5
Accuracy at step 210: 0.3
Accuracy at step 220: 0.525
Accuracy at step 230: 0.525
Accuracy at step 240: 0.65
Accuracy at step 250: 0.625
Accuracy at step 260: 0.6
Accuracy at step 270: 0.6
Accuracy at step 280: 0.425
Accuracy at step 290: 0.4
Adding run metadata for 299
Accuracy at step 300: 0.525
Accuracy at step 310: 0.375
Accuracy at step 320: 0.6
Accuracy at step 330: 0.575
Accuracy a